# 第一节 数据分析项目流程


* 此次的数据集来自在线零售业务的交易数据，该公司主要销售礼品，大部分出售对象是面向批发商。

字段介绍
* InvoiceNo: 订单编号，每笔交易有6个整数，退货订单编号开头有字母'C'。
* StockCode: 产品编号，由5个整数组成。
* Description: 产品描述。
* Quantity: 产品数量，有负号的表示退货
* InvoiceDate: 订单日期和时间。
* UnitPrice: 单价（英镑），单位产品的价格。
* CustomerID:客户编号，每个客户编号由5位数字组成。
* Country: 国家的名称，每个客户所在国家/地区的名称。

## 1. 明确问题

* 1.购买商品数前十的国家是？

* 2.交易额前十的国家是？

* 3.哪些月份销量较佳？

* 4.客单价多少？

* 5.用户消费行为分析

## 2. 理解数据 

* 获取数据

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os 
os.chdir('E:\云开明培训机构\云开见明培训课件\data summary\第六章')

In [ ]:
#导入可视化库
import plotly as py 
import plotly.graph_objs as go
py.offline.init_notebook_mode()# 初始化步骤
pyplot = py.offline.iplot  #画图函数

In [ ]:
online_data = pd.read_csv('data.csv',encoding='utf-8',dtype= {'CustomerID':str})

In [ ]:
online_data.head() #查看前5行

In [ ]:
online_data.shape#查看数据形状

In [ ]:
online_data.info()#查看数据情况

## 3 .数据清洗

In [ ]:
#统计缺失率
online_data.apply(lambda x: sum(x.isnull())/len(x), axis=0)

* 通过统计缺失率可以发现，CustomID存在部分缺失，无法进行填充，需要删除缺失的数据。订单日期和格式需要调整，方便后续分析。

In [ ]:
# 剔除缺失值
df1 = online_data.dropna(how='any').copy()  #任何一行有缺失，直接删除这一行

In [ ]:
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'], errors = 'coerce') #12/1/2010 8:26转换为2010-12-01

In [ ]:
df1.head(5)

In [ ]:
# 取用日期部分
df1['InvoiceDate'] = df1['InvoiceDate'].dt.date

* 取用日期部分或者用以下方法实现
* df1['InvoiceDate'] = df1['InvoiceDate'].apply(lambda x: x.strftime('%Y-%m-%d')) #将datetime64[ns] 转化为object
* strptime(detester,’%Y-%m-%d')#字符串转日期

In [ ]:
df1.info()

* 新建一个销售金额的字段，即用产品数量乘以单价。
* 使用apply函数，0代表按行进行计算，然后对每一列做同样的操作，1代表按列进行计算，然后对每一行做同样的操作

In [ ]:
df1['Price'] = df1.apply(lambda x: x[3] * x[5],axis=1) 

In [ ]:
# 清洗后的数据如下:

In [ ]:
df1.head(5)

## 4. 数据分析和可视化

###  第一个问题：购买商品数前十的国家是？

In [ ]:
# 获取产品数量大于0，按国家分组，对产品数量进行求和，从大到小进行排序，取前十的国家

In [ ]:
df1[df1['Quantity']>0].groupby('Country')['Quantity'].sum().sort_values(ascending=False).head(10)

In [ ]:
quantity_first_10 = df1[df1['Quantity']>0].groupby('Country')['Quantity'].sum().sort_values(ascending=False).head(10)

In [ ]:
quantity_first_10

In [ ]:
trace_basic = [go.Bar(x =quantity_first_10.index.tolist(),y =quantity_first_10.values.tolist(),
                      marker=dict(color='orange'),opacity=0.50)] #透明度
layout = go.Layout(title = '购买商品前10的国家', xaxis =dict(title ='国家'))
figure_basic = go.Figure(data = trace_basic,layout=layout)# data与layout组成一个图象对象
pyplot(figure_basic) #输出

 * 英国购买商品数远远大于其他国家，这可能由于公司位于英国，英国是该公司商品销售的主要对象。

### 第二个问题：交易额前十的国家是？

In [ ]:
# 获取产品数量大于0，按国家分组，对销售金额进行求和，从大到小进行排序

In [ ]:
trans = df1[df1['Quantity']>0].groupby('Country').sum()['Price'].sort_values(ascending=False).head(10)

In [ ]:
trace_basic = [go.Bar(x =trans.index.tolist(),y =trans.values.tolist(),
                      marker=dict(color='orange'),opacity=0.50)] #透明度
layout = go.Layout(title = '交易额前10的国家', xaxis =dict(title ='国家'))
figure_basic = go.Figure(data = trace_basic,layout=layout)# data与layout组成一个图象对象
pyplot(figure_basic) #输出

* 可以看出：英国的交易额最大，由于购买数最多，交易额理所当然也是最大。除此之外，荷兰，爱尔兰，德国，法国也是主要消费国家，需要重点关注。

###  第三个问题：哪些月份销量较佳？

In [ ]:
df1['month'] = pd.to_datetime(df1['InvoiceDate']).dt.month #取出月份数

In [ ]:
df1[df1['Quantity']>0].groupby('month')['Quantity'].sum().sort_values(ascending=False)

In [ ]:
import seaborn as sns
sns.set(style='darkgrid',context='notebook',font_scale=1.2) # 设置背景
# 支持中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
df1[df1['Quantity']>0].groupby('month')['Quantity'].sum().sort_values(ascending=False).head(12).plot(kind ='bar')
plt.xticks(rotation=45)

* 销量最高的是11月份，其次是12月份，大部分销量较佳集中在下半年，这是由于该产品是礼品，在下半年节日较多，加上促销优惠，销量比上半年月份多

###  第四个问题：客单价多少？

* 客单价即平均交易金额，是指平均每个客户交易金额，客单价等于成交金额/成交用户数。

* 销售额是由客单价和用户数所决定的，要提升产品的销售额，除了尽可能多地吸引客流，增加顾客交易次数以外，提高客单价也是非常重要的途径。

In [ ]:
sumPrice=df1[df1['Quantity']>0]['Price'].sum()
countID=df1[df1['Quantity']>0]['InvoiceNo'].count()
avgPrice = sumPrice/countID
print(avgPrice)

### 第五个问题：用户消费行为分析

* 用户消费行为分析主要从用户消费次数，用户消费金额，用户购买产品数量这三个维度进行分析。
* 首先计算每个消费者用户的消费次数，消费金额和用户购买的产品数量
* 然后在计算用户的平均情况

In [ ]:
customer = df1[df1['Quantity']>0].groupby('CustomerID').agg({'InvoiceNo': 'nunique', #统计用户不同的消费数量
                                                'Quantity': np.sum, #产品数量量
                                                'Price': np.sum})#消费总金额

In [ ]:
customer.describe()

## 5 结论和建议

通过以上计算可看出：
* 用户平均消费4次，有的客户甚至消费高达210次，是产品的忠诚客户。
* 用户平均消费金额为2053元，而75%的用户消费金额只有1661，可见，有些用户消费金额较大，属于非常有价值的用户，需要重点保持关注。
* 用户购买产品数量平均高达1194件，由于销售对象主要是批发商，算是正常的数量数。